In [1]:
import numpy as np
import pandas as pd 
import matplotlib
import seaborn as sns
import sklearn
import imblearn
import matplotlib.pyplot as plt
import time
import sklearn.metrics as m
import xgboost as xgb
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


cols = [' Bwd Packet Length Std',' PSH Flag Count',' min_seg_size_forward',' Min Packet Length',' ACK Flag Count',' Bwd Packet Length Min',' Fwd IAT Std','Init_Win_bytes_forward',' Flow IAT Max',' Bwd Packets/s',' URG Flag Count','Bwd IAT Total',' Label']
df1=pd.read_csv("/content/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv", usecols = cols)#,nrows = 50000
df2=pd.read_csv("/content/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv", usecols = cols)
df3=pd.read_csv("/content/Friday-WorkingHours-Morning.pcap_ISCX.csv", usecols = cols)

df5=pd.read_csv("/content/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv", usecols = cols)
df6=pd.read_csv("/content/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv", usecols = cols)
df7=pd.read_csv("/content/Tuesday-WorkingHours.pcap_ISCX.csv", usecols = cols)
df8=pd.read_csv("/content/Wednesday-workingHours.pcap_ISCX.csv", usecols = cols)
#df4, df7 and df8 are being left out as they only have the benign samples

In [2]:
test_data =  pd.read_excel('/content/Data_Test.xlsx', usecols = cols)

In [3]:
# test_data = test_data[test_data[' Label'] != 8]

In [4]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0    Bwd Packet Length Min  53 non-null     int64  
 1    Bwd Packet Length Std  53 non-null     float64
 2    Flow IAT Max           53 non-null     int64  
 3    Fwd IAT Std            53 non-null     float64
 4   Bwd IAT Total           53 non-null     int64  
 5    Bwd Packets/s          53 non-null     float64
 6    Min Packet Length      53 non-null     int64  
 7    PSH Flag Count         53 non-null     int64  
 8    ACK Flag Count         53 non-null     int64  
 9    URG Flag Count         53 non-null     int64  
 10  Init_Win_bytes_forward  53 non-null     int64  
 11   min_seg_size_forward   53 non-null     int64  
 12   Label                  53 non-null     object 
dtypes: float64(3), int64(9), object(1)
memory usage: 5.5+ KB


In [5]:
df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
# df = pd.concat([df,df4])
# del df4
df = pd.concat([df,df5])
del df5
df = pd.concat([df,df6])
del df6
# df = pd.concat([df,df7])
# del df7
# df = pd.concat([df,df8])
# del df8

data = df.copy()

def process_data(data):
    for column in data.columns:
        if data[column].dtype == np.int64:
            maxVal = data[column].max()
            if maxVal < 120:
                data[column] = data[column].astype(np.int8)
            elif maxVal < 32767:
                data[column] = data[column].astype(np.int16)
            else:
                data[column] = data[column].astype(np.int32)

        if data[column].dtype == np.float64:
            maxVal = data[column].max()
            minVal = data[data[column] > 0][column].min()
            if maxVal < 120 and minVal > 0.01:
                data[column] = data[column].astype(np.float16)
            else:
                data[column] = data[column].astype(np.float32)

    attackType = data[' Label'].unique()
    data[' Label'] = data[' Label'].astype('category')
    data[' Label'] = data[' Label'].astype("category").cat.codes

    return data
process_data(data)
process_data(test_data)

,Bwd Packet Length Min,Bwd Packet Length Std,Flow IAT Max,Fwd IAT Std,Bwd IAT Total,Bwd Packets/s,Min Packet Length,PSH Flag Count,ACK Flag Count,URG Flag Count,Init_Win_bytes_forward,min_seg_size_forward,Label
0,86,0.000000,16833,0.000000e+00,17320,103.723679,45,0,0,0,-1,8,6
1,0,430.098602,953312,3.987668e+05,2045420,3.119674,0,1,0,0,64240,20,6
2,49,0.000000,1270,0.000000e+00,0,787.401550,33,0,0,0,-1,8,6
3,73,0.000000,22214,0.000000e+00,22612,84.860825,45,0,0,0,-1,8,6
4,0,616.068726,454835,6.967055e+04,835775,8.201840,0,1,0,0,8192,20,6
5,67,0.000000,1116,0.000000e+00,0,896.057373,35,0,0,0,-1,8,6
6,0,430.098602,953197,3.891110e+05,1566101,4.179067,0,1,0,0,8192,20,6
7,97,0.000000,955,0.000000e+00,0,1047.120361,34,0,0,0,-1,8,6
8,136,0.000000,595,0.000000e+00,0,1680.672241,51,0,0,0,-1,8,6
9,100,0.000000,1093,0.000000e+00,0,914.913086,40,0,0,0,-1,8,0


In [6]:
data.head()

,Bwd Packet Length Min,Bwd Packet Length Std,Flow IAT Max,Fwd IAT Std,Bwd IAT Total,Bwd Packets/s,Min Packet Length,PSH Flag Count,ACK Flag Count,URG Flag Count,Init_Win_bytes_forward,min_seg_size_forward,Label
0,0,0.0,3,0.0,0,0.000000,6,0,1,0,33,20,0
1,6,0.0,109,0.0,0,9174.311523,6,0,1,1,29,20,0
2,6,0.0,52,0.0,0,19230.769531,6,0,1,1,29,20,0
3,6,0.0,34,0.0,0,29411.765625,6,0,1,1,31,20,0
4,0,0.0,3,0.0,0,0.000000,6,0,1,0,32,20,0


In [7]:
shuffled_data = data.sample(frac=1)  
export_data = shuffled_data.tail(100) 


export_data.to_csv('exported_data.csv', index=False)

In [8]:
y = data[' Label'].copy()
X = data.drop([' Label'],axis=1)

In [9]:
y.value_counts()

0    871074
4    158930
2    128027
1      1966
5      1507
7       652
3        36
6        21
Name:  Label, dtype: int64

In [10]:
y_test_data = test_data[' Label'].copy()
X_test_data = test_data.drop([' Label'],axis=1)

In [11]:
y_test_data.value_counts()

2    12
6     9
0     8
4     6
3     6
7     5
1     3
8     2
5     2
Name:  Label, dtype: int64

In [12]:
X_test_data.columns

Index([' Bwd Packet Length Min', ' Bwd Packet Length Std', ' Flow IAT Max',
       ' Fwd IAT Std', 'Bwd IAT Total', ' Bwd Packets/s', ' Min Packet Length',
       ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count',
       'Init_Win_bytes_forward', ' min_seg_size_forward'],
      dtype='object')

In [13]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler('majority')
X_rus, y_rus = rus.fit_sample(X, y)

In [14]:
y_rus.value_counts()

4    158930
2    128027
1      1966
5      1507
7       652
3        36
6        21
0        21
Name:  Label, dtype: int64

In [15]:
df = X_rus
df[' Label'] = y_rus
minor = pd.DataFrame(df[(df[' Label']!=4) & (df[' Label']!=2)])
major = pd.DataFrame(df[(df[' Label']==4) | (df[' Label']==2)])
minor[' Label'].value_counts()

1    1966
5    1507
7     652
3      36
6      21
0      21
Name:  Label, dtype: int64

In [16]:
from imblearn.over_sampling import SMOTE
y_rus_ =  minor[' Label']
X_rus_ =  minor.drop([' Label'],axis=1)
strategy = {1:2000, 5:1600, 7:800, 3:300, 6:200, 0:200}
# strategy = {10: 158930, 3: 20000, 7: 16000, 11: 12000, 6: 12000, 5: 12000, 1: 12000, 12: 8000, 14: 5000, 9: 3000, 13: 2000, 8: 2000, 0: 2000}
sm = SMOTE(sampling_strategy=strategy)
X_sm, y_sm = sm.fit_sample(X_rus_, y_rus_)
X_min,y_min = X_sm, y_sm 

In [17]:
major[' Label'].value_counts()

4    158930
2    128027
Name:  Label, dtype: int64

In [18]:
from imblearn.under_sampling import RandomUnderSampler
y_rus_ =  major[' Label']
X_rus_ =  major.drop([' Label'],axis=1)
strategy = {4:10000, 2:6000}
tom = RandomUnderSampler(sampling_strategy=strategy)
X_tom, y_tom = tom.fit_sample(X_rus_, y_rus_)
y_tom.value_counts()

4    10000
2     6000
Name:  Label, dtype: int64

In [19]:
X.head(1)

,Bwd Packet Length Min,Bwd Packet Length Std,Flow IAT Max,Fwd IAT Std,Bwd IAT Total,Bwd Packets/s,Min Packet Length,PSH Flag Count,ACK Flag Count,URG Flag Count,Init_Win_bytes_forward,min_seg_size_forward
0,0,0.0,3,0.0,0,0.0,6,0,1,0,33,20


In [20]:
X_maj,y_maj = X_tom, y_tom
X,y = pd.concat([X_maj,X_min]), pd.concat([y_maj,y_min])
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21100 entries, 0 to 5099
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0    Bwd Packet Length Min  21100 non-null  int16  
 1    Bwd Packet Length Std  21100 non-null  float32
 2    Flow IAT Max           21100 non-null  int32  
 3    Fwd IAT Std            21100 non-null  float32
 4   Bwd IAT Total           21100 non-null  int32  
 5    Bwd Packets/s          21100 non-null  float32
 6    Min Packet Length      21100 non-null  int16  
 7    PSH Flag Count         21100 non-null  int8   
 8    ACK Flag Count         21100 non-null  int8   
 9    URG Flag Count         21100 non-null  int8   
 10  Init_Win_bytes_forward  21100 non-null  int32  
 11   min_seg_size_forward   21100 non-null  int8   
dtypes: float32(3), int16(2), int32(3), int8(4)
memory usage: 824.2 KB


In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [22]:
# extract numerical attributes and scale it to have zero mean and unit variance  
cols_test = X_test_data.select_dtypes(include=['float32','float16','int32','int16','int8']).columns
X_test_data = scaler.fit_transform(X_test_data.select_dtypes(include=['float32','float16','int32','int16','int8']))

In [23]:
y_test_data.value_counts()

2    12
6     9
0     8
4     6
3     6
7     5
1     3
8     2
5     2
Name:  Label, dtype: int64

In [24]:


# extract numerical attributes and scale it to have zero mean and unit variance  
cols = X.select_dtypes(include=['float32','float16','int32','int16','int8']).columns
train_X = scaler.fit_transform(X.select_dtypes(include=['float32','float16','int32','int16','int8']))



from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(train_X,y,train_size=0.70, random_state=2)


from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Train Random Forest
RFC_Classifier = RandomForestClassifier(max_depth=40)
RFC_Classifier.fit(X_train, Y_train)
print ('RF Classifier run')

# Train SVC
SVM_Classifier = SVC()
SVM_Classifier.fit(X_train, Y_train)
print ('SV Classifier run')
# Train Decision Tree Model
DTC_Classifier = tree.DecisionTreeClassifier(criterion='gini', max_depth=33, random_state=20, max_features=12, splitter='random')
DTC_Classifier.fit(X_train, Y_train)
print ('DTC Classifier run')

RF Classifier run
SV Classifier run
DTC Classifier run


In [25]:
from sklearn import metrics

models = []
models.append(('Random Forest Classifier', RFC_Classifier))
models.append(('Decision Tree Classifier', DTC_Classifier))
models.append(('Support Vector Classifier',SVM_Classifier))


for i, v in models:
    Xpred =  v.predict(X_train)
    scores = cross_val_score(v, X_train, Y_train, cv=10)
    accuracy = metrics.accuracy_score(Y_train, Xpred)
    confusion_matrix = metrics.confusion_matrix(Y_train, Xpred)
    classification = metrics.classification_report(Y_train, Xpred)
    print()
    print('============================== {} Model Evaluation =============================='.format(i))
    print()
    print ("Cross Validation Mean Score:" "\n", scores.mean())
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()


============================== Random Forest Classifier Model Evaluation ==============================

Cross Validation Mean Score:
 0.9617443987574609

Model Accuracy:
 0.9897758819148216

Confusion matrix:
 [[ 140    0    0    0    0    0    0    0]
 [   0 1359    0    0    0    0    0    0]
 [   0    0 4232    0    0    0    0    0]
 [   0    0    0  197    0    0    0    0]
 [   0    0    0    0 7034    0    0    0]
 [   0    0    0    0    0 1105    0    0]
 [   0    0    0    0    0    1  141    0]
 [   0    0    0    0    0  150    0  410]]

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       140
           1       1.00      1.00      1.00      1359
           2       1.00      1.00      1.00      4232
           3       1.00      1.00      1.00       197
           4       1.00      1.00      1.00      7034
           5       0.88      1.00      0.94      1105
           6       1.00      0.99      

In [26]:
for i, v in models:
    pred = v.predict(X_test)
    accuracy = metrics.accuracy_score(Y_test,pred)
    confusion_matrix = metrics.confusion_matrix(Y_test, pred)
    classification = metrics.classification_report(Y_test, pred)
    print()
    print('============================== {} Model Test Results =============================='.format(i))
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()    


============================== Random Forest Classifier Model Test Results ==============================

Model Accuracy:
 0.9638287790238509

Confusion matrix:
 [[  59    0    0    1    0    0    0    0]
 [   0  640    1    0    0    0    0    0]
 [   0    2 1766    0    0    0    0    0]
 [   0    1    0  102    0    0    0    0]
 [   0    0    0    0 2966    0    0    0]
 [   0    0    0    0    0  425    0   70]
 [   0    0    0    0    0    2   56    0]
 [   1    0    0    0    0  148    3   88]]

Classification report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98        60
           1       1.00      1.00      1.00       641
           2       1.00      1.00      1.00      1768
           3       0.99      0.99      0.99       103
           4       1.00      1.00      1.00      2966
           5       0.74      0.86      0.79       495
           6       0.95      0.97      0.96        58
           7       0.56      0.37

In [27]:
from sklearn.ensemble import VotingClassifier

clf1 = tree.DecisionTreeClassifier(criterion='gini', max_depth=33, random_state=20, max_features=12, splitter='random')
clf2 = RandomForestClassifier(criterion='gini', max_depth=40, random_state=20)
clf3 = SVC()

votingC = VotingClassifier(estimators=[('dc',clf1), ('rf', clf2),('svc',clf3)],voting='hard', weights=[2,2,1],flatten_transform=True)
votingC.fit(X_train,Y_train)

VotingClassifier(estimators=[('dc',
                              DecisionTreeClassifier(max_depth=33,
                                                     max_features=12,
                                                     random_state=20,
                                                     splitter='random')),
                             ('rf',
                              RandomForestClassifier(max_depth=40,
                                                     random_state=20)),
                             ('svc', SVC())],
                 weights=[2, 2, 1])

In [28]:
import joblib
# Save the model to a file
joblib.dump(votingC, 'ensemble_model.pkl')

['ensemble_model.pkl']

In [29]:
pred = votingC.predict(X_test)
accuracy = metrics.accuracy_score(Y_test,pred)
confusion_matrix = metrics.confusion_matrix(Y_test, pred)
classification = metrics.classification_report(Y_test, pred)
print()
print('============================== {} Model Test Results =============================='.format('Voting Classifier'))
print()
print ("Model Accuracy:" "\n", accuracy)
print()
print("Confusion matrix:" "\n", confusion_matrix)
print()
print("Classification report:" "\n", classification) 
print()    


============================== Voting Classifier Model Test Results ==============================

Model Accuracy:
 0.9638287790238509

Confusion matrix:
 [[  58    2    0    0    0    0    0    0]
 [   0  641    0    0    0    0    0    0]
 [   0    0 1768    0    0    0    0    0]
 [   0    1    0  102    0    0    0    0]
 [   0    0    0    0 2966    0    0    0]
 [   0    0    0    0    0  432    2   61]
 [   0    0    0    0    0    1   57    0]
 [   1    0    0    0    0  160    1   78]]

Classification report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97        60
           1       1.00      1.00      1.00       641
           2       1.00      1.00      1.00      1768
           3       1.00      0.99      1.00       103
           4       1.00      1.00      1.00      2966
           5       0.73      0.87      0.79       495
           6       0.95      0.98      0.97        58
           7       0.56      0.33      0

In [30]:
pred_test = votingC.predict(X_test_data)
accuracy = metrics.accuracy_score(y_test_data,pred_test)
confusion_matrix = metrics.confusion_matrix(y_test_data, pred_test)
classification = metrics.classification_report(y_test_data, pred_test)
print()
print('============================== {} Model Test Results =============================='.format('Voting Classifier'))
print()
print ("Model Accuracy:" "\n", accuracy)
print()
print("Confusion matrix:" "\n", confusion_matrix)
print()
print("Classification report:" "\n", classification) 
print()


============================== Voting Classifier Model Test Results ==============================

Model Accuracy:
 0.2830188679245283

Confusion matrix:
 [[3 0 4 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 2 0]
 [6 0 6 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 5 0]
 [0 0 0 0 6 0 0 0 0]
 [0 2 0 0 0 0 0 0 0]
 [6 0 3 0 0 0 0 0 0]
 [0 0 0 0 5 0 0 0 0]
 [0 2 0 0 0 0 0 0 0]]

Classification report:
               precision    recall  f1-score   support

           0       0.19      0.38      0.25         8
           1       0.00      0.00      0.00         3
           2       0.43      0.50      0.46        12
           3       0.00      0.00      0.00         6
           4       0.55      1.00      0.71         6
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         9
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         2

    accuracy                           0.28        53
   macro avg       0.13      0.2

In [31]:
from sklearn import metrics

models = []
models.append(('Random Forest Classifier', RFC_Classifier))
models.append(('Decision Tree Classifier', DTC_Classifier))
models.append(('Support Vector Classifier',SVM_Classifier))


for i, v in models:
    Xpred =  v.predict(X_test_data)
    scores = cross_val_score(v, X_test_data, y_test_data, cv=10)
    accuracy = metrics.accuracy_score(y_test_data, Xpred)
    confusion_matrix = metrics.confusion_matrix(y_test_data, Xpred)
    classification = metrics.classification_report(y_test_data, Xpred)
    print()
    print('============================== {} Model Evaluation =============================='.format(i))
    print()
    print ("Cross Validation Mean Score:" "\n", scores.mean())
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()


============================== Random Forest Classifier Model Evaluation ==============================

Cross Validation Mean Score:
 0.7733333333333333

Model Accuracy:
 0.16981132075471697

Confusion matrix:
 [[3 0 1 0 3 0 0 1 0]
 [1 0 0 0 0 0 0 2 0]
 [6 0 0 0 6 0 0 0 0]
 [0 0 0 0 1 0 0 5 0]
 [0 0 0 0 6 0 0 0 0]
 [0 2 0 0 0 0 0 0 0]
 [7 0 2 0 0 0 0 0 0]
 [0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 0 2 0 0]]

Classification report:
               precision    recall  f1-score   support

           0       0.18      0.38      0.24         8
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         6
           4       0.29      1.00      0.44         6
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         9
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         2

    accuracy                  

In [32]:
model = joblib.load('/content/ensemble_model.pkl')

In [33]:

sample = X_test_data[11].reshape(1, -1)

prediction = v.predict(sample)

print('Prediction:', prediction)

Prediction: [2]


In [34]:
Xpred =  v.predict(X_test_data)